# POD from pointclouds 

We will now proceed to explain how to perform POD from point clouds. In this instance, we test only for POD in serial, as to perform in parallel, a parallel reader/writer is needed.

If you have saved information in hdf5 and have habilitated mpi4py compilation of it, then you could use this code in parallel.

#### Import general modules

In [1]:
# Import required modules
from mpi4py import MPI #equivalent to the use of MPI_init() in C
import matplotlib.pyplot as plt
import numpy as np
import h5py

# Get mpi info
comm = MPI.COMM_WORLD

## Set up the input parameters

In [2]:
file_sequence = [f"interpolated_fields{str(1+i).zfill(5)}.hdf5" for i in range(0, 48)]
pod_fields = ["u", "v", "w"]
mesh_fname = "points.hdf5"
mass_matrix_fname = "points.hdf5"
mass_matrix_key = "mass"
k = 10
p = 10

# Calculated parameters
number_of_pod_fields = len(pod_fields)

## Call the pynektools routines

In [3]:
# Import IO helper functions
from pynektools.io.utils import get_fld_from_ndarray

# Import types asociated with POD
from pynektools.rom.pod import POD
from pynektools.rom.io_help import IoHelp

# Output
from pyevtk.hl import gridToVTK

# Load the mesh
with h5py.File(mesh_fname, 'r') as f:
    x = f["x"][:]
    y = f["y"][:]
    z = f["z"][:]

# Load the mass matrix
with h5py.File(mass_matrix_fname, 'r') as f:
    bm = f[mass_matrix_key][:]
bm[np.where(bm == 0)] = 1e-8

# Instance io helper that will serve as buffer for the snapshots
ioh = IoHelp(comm, number_of_fields = number_of_pod_fields, batch_size = p, field_size = bm.size)

# Put the mass matrix in the appropiate format (long 1d array)
mass_list = []
for i in range(0, number_of_pod_fields):
    mass_list.append(np.copy(np.sqrt(bm)))
ioh.copy_fieldlist_to_xi(mass_list)
ioh.bm1sqrt[:,:] = np.copy(ioh.xi[:,:])

# Instance the POD object
pod = POD(comm, number_of_modes_to_update = k, global_updates = True, auto_expand = False)

# Perform reading and updates
j = 0
while j < len(file_sequence):

    # Load the snapshot data
    fname = file_sequence[j]
    with h5py.File(fname, 'r') as f:
        fld_data = []
        for field in pod_fields:
            fld_data.append(f[field][:])

    # Put the snapshot data into a column array
    ioh.copy_fieldlist_to_xi(fld_data)

    # Load the column array into the buffer
    ioh.load_buffer(scale_snapshot = True)
    
    # Update POD modes
    if ioh.update_from_buffer:
        pod.update(comm, buff = ioh.buff[:,:(ioh.buffer_index)])

    j += 1

# Check if there is information in the buffer that should be taken in case the loop exit without flushing
if ioh.buffer_index > ioh.buffer_max_index:
    ioh.log.write("info","All snapshots where properly included in the updates")
else: 
    ioh.log.write("warning","Last loaded snapshot to buffer was: "+repr(ioh.buffer_index-1))
    ioh.log.write("warning","The buffer updates when it is full to position: "+repr(ioh.buffer_max_index))
    ioh.log.write("warning","Data must be updated now to not lose anything,  Performing an update with data in buffer ")
    pod.update(comm, buff = ioh.buff[:,:(ioh.buffer_index)])

# Scale back the modes
pod.scale_modes(comm, bm1sqrt = ioh.bm1sqrt, op = "div")

# Rotate local modes back to global, This only enters in effect if global_update = false
pod.rotate_local_modes_to_global(comm)

# Write the data in vtk for now
# Go over the modes
for j in range(0, k):
    
    ## Split the snapshots into the proper fields
    field_list1d = ioh.split_narray_to_1dfields(pod.u_1t[:,j])
    field_dict = {}
    for i in range(0, len(pod_fields)):
        field_dict[f"{pod_fields[i]}_mode"] = field_list1d[i].reshape(bm.shape)

    # write to vtk
    gridToVTK( "pod_mode"+str(j).zfill(5),  x, y, z, pointData=field_dict)

2024-09-19 17:11:54,665 - io_helper - INFO - io_helper object initialized
2024-09-19 17:11:54,670 - pod - WARNING - ifget_all_modes is hard coded to False. This parameter applies to lcl updates. It controls if one gets all modes in the global rotation, despite keeping less modes locally. I do not see a use for this in production runs. Thus it is set to false. If needed, activate in mpi_spSVD.py module
2024-09-19 17:11:54,671 - pod - INFO - POD Object initialized
2024-09-19 17:11:54,681 - io_helper - INFO - Loaded snapshot in buffer in pos: 0
2024-09-19 17:11:54,687 - io_helper - INFO - Loaded snapshot in buffer in pos: 1
2024-09-19 17:11:54,693 - io_helper - INFO - Loaded snapshot in buffer in pos: 2
2024-09-19 17:11:54,700 - io_helper - INFO - Loaded snapshot in buffer in pos: 3
2024-09-19 17:11:54,706 - io_helper - INFO - Loaded snapshot in buffer in pos: 4
2024-09-19 17:11:54,712 - io_helper - INFO - Loaded snapshot in buffer in pos: 5
2024-09-19 17:11:54,718 - io_helper - INFO - Lo